In [2]:
!pip install shapely
!pip install matplotlib
!pip install seaborn

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
#Importar un archivo CSV

# Cargar el archivo CSV
df_csv = pd.read_csv("electric_vehicle_data.csv")

# Mostrar las primeras filas del archivo importado
print(df_csv.head())

# Mostrar la estructura del DataFrame
print(df_csv.info())

# Mostrar un resumen estadístico de las variables numéricas
print(df_csv.describe())

   VIN (1-10)    County     City State  Postal Code  Model Year     Make  \
0  5YJ3E1EBXK      King  Seattle    WA      98178.0        2019    TESLA   
1  5YJYGDEE3L    Kitsap  Poulsbo    WA      98370.0        2020    TESLA   
2  KM8KRDAF5P    Kitsap   Olalla    WA      98359.0        2023  HYUNDAI   
3  5UXTA6C0XM    Kitsap  Seabeck    WA      98380.0        2021      BMW   
4  JTMAB3FV7P  Thurston  Rainier    WA      98576.0        2023   TOYOTA   

        Model                   Electric Vehicle Type  \
0     MODEL 3          Battery Electric Vehicle (BEV)   
1     MODEL Y          Battery Electric Vehicle (BEV)   
2     IONIQ 5          Battery Electric Vehicle (BEV)   
3          X5  Plug-in Hybrid Electric Vehicle (PHEV)   
4  RAV4 PRIME  Plug-in Hybrid Electric Vehicle (PHEV)   

   Clean Alternative Fuel Vehicle (CAFV) Eligibility  Electric Range  \
0            Clean Alternative Fuel Vehicle Eligible           220.0   
1            Clean Alternative Fuel Vehicle Eligible    

In [5]:
# VIN debe ser el índice de DataFrame

#Lo solucionamos: 

df_csv = df_csv.set_index("VIN (1-10)")

# Confirme si el índice se configuró correctamente
print(df_csv.head())

print(df_csv.describe())

              County     City State  Postal Code  Model Year     Make  \
VIN (1-10)                                                              
5YJ3E1EBXK      King  Seattle    WA      98178.0        2019    TESLA   
5YJYGDEE3L    Kitsap  Poulsbo    WA      98370.0        2020    TESLA   
KM8KRDAF5P    Kitsap   Olalla    WA      98359.0        2023  HYUNDAI   
5UXTA6C0XM    Kitsap  Seabeck    WA      98380.0        2021      BMW   
JTMAB3FV7P  Thurston  Rainier    WA      98576.0        2023   TOYOTA   

                 Model                   Electric Vehicle Type  \
VIN (1-10)                                                       
5YJ3E1EBXK     MODEL 3          Battery Electric Vehicle (BEV)   
5YJYGDEE3L     MODEL Y          Battery Electric Vehicle (BEV)   
KM8KRDAF5P     IONIQ 5          Battery Electric Vehicle (BEV)   
5UXTA6C0XM          X5  Plug-in Hybrid Electric Vehicle (PHEV)   
JTMAB3FV7P  RAV4 PRIME  Plug-in Hybrid Electric Vehicle (PHEV)   

            Clean Alterna

In [6]:
#  Detectar y tratar los datos ausentes es una parte crucial en el proceso de limpieza de datos. 
# Detectar valores nulos en todo el DataFrame
print(df_csv.isnull().sum())

County                                                 3
City                                                   3
State                                                  0
Postal Code                                            3
Model Year                                             0
Make                                                   0
Model                                                  0
Electric Vehicle Type                                  0
Clean Alternative Fuel Vehicle (CAFV) Eligibility      0
Electric Range                                        36
Base MSRP                                             36
Legislative District                                 494
DOL Vehicle ID                                         0
Vehicle Location                                      10
Electric Utility                                       3
2020 Census Tract                                      3
dtype: int64


In [7]:
#Copia del csv por si queremos recuperar después
df_csv2 = df_csv.copy()


In [8]:

#Eliminamos los valores nulos
df_csv2= df_csv2.dropna()
#Comprobamos que se hallan borrandonoslas los campos nulos y a partir de ahora trabajamos con la copia
print(df_csv2.isnull().sum())

County                                               0
City                                                 0
State                                                0
Postal Code                                          0
Model Year                                           0
Make                                                 0
Model                                                0
Electric Vehicle Type                                0
Clean Alternative Fuel Vehicle (CAFV) Eligibility    0
Electric Range                                       0
Base MSRP                                            0
Legislative District                                 0
DOL Vehicle ID                                       0
Vehicle Location                                     0
Electric Utility                                     0
2020 Census Tract                                    0
dtype: int64


In [9]:
# Comprobar el tipo de dato de cada columna 
df_csv2.dtypes

County                                                object
City                                                  object
State                                                 object
Postal Code                                          float64
Model Year                                             int64
Make                                                  object
Model                                                 object
Electric Vehicle Type                                 object
Clean Alternative Fuel Vehicle (CAFV) Eligibility     object
Electric Range                                       float64
Base MSRP                                            float64
Legislative District                                 float64
DOL Vehicle ID                                         int64
Vehicle Location                                      object
Electric Utility                                      object
2020 Census Tract                                    float64
dtype: object

In [10]:
#Verificar el tipo de datos de "Vehicle Location"
print(type(df_csv2['Vehicle Location'].iloc[0]))

#Conversión a "Point"
from shapely.wkt import loads as wkt_loads

df_csv2['Vehicle Location'] = df_csv2['Vehicle Location'].apply(
    lambda x: wkt_loads(x) if pd.notnull(x) else None
)

print(type(df_csv2['Vehicle Location'].iloc[0]))

<class 'str'>
<class 'shapely.geometry.point.Point'>


In [11]:

#Cambiar el tipo de CP a string en vez de float
df_csv2['Postal Code'] = df_csv2['Postal Code'].apply(
    lambda x: str(int(x)) if pd.notnull(x) else ''
)

#El Census Tract es una cadena de caracteres identificativos de una subdivision de un condado de EEUU, 
#que al igual que el CP, no haremos operaciones matematicas sobre ella por lo que lo pasamos como string
df_csv2['2020 Census Tract'] = df_csv2['2020 Census Tract'].apply(
    lambda x: str(int(x)) if pd.notnull(x) else ''
)

df_csv2['DOL Vehicle ID'] = df_csv2['DOL Vehicle ID'].apply(
    lambda x: str(int(x)) if pd.notnull(x) else ''
)

#Cambiar a enterno sin decimales
df_csv2['Electric Range'] = df_csv2['Electric Range'].apply(lambda x: int(x) if pd.notnull(x) else None)
df_csv2['Base MSRP'] = df_csv2['Base MSRP'].apply(lambda x: int(x) if pd.notnull(x) else None)


In [12]:
# Comprobar el tipo de dato de cada columna 
df_csv2.dtypes

County                                                object
City                                                  object
State                                                 object
Postal Code                                           object
Model Year                                             int64
Make                                                  object
Model                                                 object
Electric Vehicle Type                                 object
Clean Alternative Fuel Vehicle (CAFV) Eligibility     object
Electric Range                                         int64
Base MSRP                                              int64
Legislative District                                 float64
DOL Vehicle ID                                        object
Vehicle Location                                      object
Electric Utility                                      object
2020 Census Tract                                     object
dtype: object

In [13]:

# Mostrar las primeras filas del archivo importado
print(df_csv2.head())

              County     City State Postal Code  Model Year     Make  \
VIN (1-10)                                                             
5YJ3E1EBXK      King  Seattle    WA       98178        2019    TESLA   
5YJYGDEE3L    Kitsap  Poulsbo    WA       98370        2020    TESLA   
KM8KRDAF5P    Kitsap   Olalla    WA       98359        2023  HYUNDAI   
5UXTA6C0XM    Kitsap  Seabeck    WA       98380        2021      BMW   
JTMAB3FV7P  Thurston  Rainier    WA       98576        2023   TOYOTA   

                 Model                   Electric Vehicle Type  \
VIN (1-10)                                                       
5YJ3E1EBXK     MODEL 3          Battery Electric Vehicle (BEV)   
5YJYGDEE3L     MODEL Y          Battery Electric Vehicle (BEV)   
KM8KRDAF5P     IONIQ 5          Battery Electric Vehicle (BEV)   
5UXTA6C0XM          X5  Plug-in Hybrid Electric Vehicle (PHEV)   
JTMAB3FV7P  RAV4 PRIME  Plug-in Hybrid Electric Vehicle (PHEV)   

            Clean Alternative Fu

In [17]:
# Identificar datos atípicos y su tratamiento para evitar que puedan distorsionar futuros análisis estadísticos


# Parte I: Variables numéricas

# Seleccionamos solo las columnas numéricas
numerical_cols = df_csv2.select_dtypes(include=[np.number]).columns
print(numerical_cols)

Index(['Model Year', 'Electric Range', 'Base MSRP', 'Legislative District'], dtype='object')


In [15]:
# Usamos el rango intercuartílico (IQR)
# IQR es la diferencia entre el tercer cuartil (Q3) y el primer cuartil (Q1). Los valores que caen fuera de este rango son considerados atípicos.

def identify_outliers_iqr(df, numerical_cols):
    outliers = {}
    for col in numerical_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Identificar los outliers
        outliers[col] = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    return outliers

# Identificar outliers usando el método IQR
outliers_iqr = identify_outliers_iqr(df_csv2, numerical_cols)
print("\nDatos atípicos identificados por IQR:")
for col, data in outliers_iqr.items():
    print(f"{col}: {data.shape[0]} outliers")


Datos atípicos identificados por IQR:
Model Year: 6445 outliers
Electric Range: 38660 outliers
Base MSRP: 3241 outliers
Legislative District: 0 outliers


In [22]:

print("Mínimo Model Year:", df_csv2['Model Year'].min())
print("Máximo Model Year:", df_csv2['Model Year'].max())
print("\n")

print("Mínimo Electric Range:", df_csv2['Electric Range'].min())
print("Máximo Electric Range:", df_csv2['Electric Range'].max())
print("\n")

#No vamos a eliminar los datos atípicos del MSRP porque se desconocen muchos datos 
print("Precio de venta sugerido por el fabricante:")
print("Mínimo Base MSRP:", df_csv2['Base MSRP'].min())
print("Máximo Base MSRP:", df_csv2['Base MSRP'].max())


Mínimo Model Year: 2000
Máximo Model Year: 2025


Mínimo Electric Range: 0
Máximo Electric Range: 337


Precio de venta sugerido por el fabricante:
Mínimo Base MSRP: 0
Máximo Base MSRP: 845000


In [ ]:
# Parte II: Variables NO Numéricas 

non_numeric_columns = df_csv2.select_dtypes(exclude=[np.number]).columns

# Columnas a eliminar
columns_to_exclude = ['Postal Code', 'City', 'State', 'Model']

# Filtrar
non_numeric_columns = [col for col in non_numeric_columns if col not in columns_to_exclude]

In [ ]:
# Generar gráficos para cada columna no numérica
for col in non_numeric_columns:
    # Contar la frecuencia de las categorías
    plt.figure(figsize=(10, 6))
    sns.countplot(x=col, data=df_csv2, palette="Set2")
    
    # Título y formato
    plt.title(f'Frecuencia de las categorías en {col}', fontsize=16)
    plt.xticks(rotation=45, ha='right', fontsize=12)
    plt.yticks(fontsize=12)
    
    # Etiquetas y formato del gráfico
    plt.xlabel(col, fontsize=14)
    plt.ylabel('Frecuencia', fontsize=14)
    
    # Mostrar el gráfico
    plt.tight_layout()
    plt.show()
